In [ ]:
#SIMILITUD DE COSENOS

In [6]:
#importamos las librerias que se vana autilizar
import numpy as np
from pyspark.ml.linalg import *
from pyspark.sql.types import * 
from pyspark.sql.functions import *

#funcion para generar un vestor random
def random_dense_vector(length=10):
    return Vectors.dense([float(np.random.random()) for i in range(length)])

#se crea un vector random estatico
static_vector = random_dense_vector()


df = spark.createDataFrame([[random_dense_vector()] for x in range(10)], ["myCol"])
df.limit(3).toPandas()

#similaridad de coseno
def cos_sim(a,b):
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

#aplicamos udf a las columnas
df = df.withColumn("coSim", udf(cos_sim, FloatType())(col("myCol"), array([lit(v) for v in static_vector])))
df.limit(10).toPandas()

,myCol,coSim
0,"[0.7522716397235971, 0.08349043928935196, 0.29...",0.675685
1,"[0.7444252035889254, 0.07485319151376468, 0.13...",0.669950
2,"[0.3455906004548336, 0.7298718514822977, 0.162...",0.837654
3,"[0.861522590811416, 0.758465432129017, 0.14662...",0.696734
4,"[0.8935267690156283, 0.28618536782972026, 0.17...",0.739941
5,"[0.8641126563384082, 0.1080635160900919, 0.074...",0.757619
6,"[0.7436462825553632, 0.1383844441626948, 0.745...",0.783198
7,"[0.6754186114954661, 0.24416162502123773, 0.87...",0.734867
8,"[0.9723202084183448, 0.8516161700303959, 0.964...",0.661223
9,"[0.292144978856558, 0.06118236594555526, 0.342...",0.626082


In [3]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/09 12:19:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/09 12:19:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/09 12:19:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [ ]:
#Escalado

In [5]:
df = spark.createDataFrame([ (1, 'H',12560,45),
                             (1, 'O',42560,90),
                             (1, 'L',31285,120),
                             (1, 'I',10345,150)
                           ], ["userID", "Name","Revenue","No_of_Days"])

print("Antes :")
df.show(5)

from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# UDF para convertir el tipo de columna de vector a tipo double

unlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())

#Iterando columnas para escalar
for i in ["Revenue","No_of_Days"]:
    # VectorAssembler transformacion
    assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")

    #Transformacion MinMaxScaler
    scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")

    # Canalizacion de VectorAssembler y MinMaxScaler
    pipeline = Pipeline(stages=[assembler, scaler])

    # Ajuste en el marco de datos 
    df = pipeline.fit(df).transform(df).withColumn(i+"_Scaled", unlist(i+"_Scaled")).drop(i+"_Vect")

print("Despues :")
df.show(5)

Antes :
+------+----+-------+----------+
|userID|Name|Revenue|No_of_Days|
+------+----+-------+----------+
|     1|   H|  12560|        45|
|     1|   O|  42560|        90|
|     1|   L|  31285|       120|
|     1|   I|  10345|       150|
+------+----+-------+----------+

Despues :
+------+----+-------+----------+--------------+-----------------+
|userID|Name|Revenue|No_of_Days|Revenue_Scaled|No_of_Days_Scaled|
+------+----+-------+----------+--------------+-----------------+
|     1|   H|  12560|        45|         0.069|              0.0|
|     1|   O|  42560|        90|           1.0|            0.429|
|     1|   L|  31285|       120|          0.65|            0.714|
|     1|   I|  10345|       150|           0.0|              1.0|
+------+----+-------+----------+--------------+-----------------+

